In [ ]:
import pickle
import os
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
import seaborn as sns
from transformers import GPT2Tokenizer, GPT2Config
from LLMHeadModelWithFFNOutput import LLMHeadModelWithFFNOutput
from matplotlib import pyplot as plt
from sklearn.decomposition import NMF
from sklearn.preprocessing import MinMaxScaler

In [51]:
# Configuración del modelo y el tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
config = GPT2Config.from_pretrained('gpt2')
num_layers = config.n_layer
num_neurons = config.hidden_size * 4
max_length = 19
index_number = -3
test_file = os.path.join('datasets', 'test_dataset.txt')
folder = os.path.join('experiments', "activations")
file = os.path.join(folder, "gpt2.pickle")
model = LLMHeadModelWithFFNOutput.from_pretrained('gpt2-finetuned', config=config)
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("Device:", device)

Device: cuda


In [52]:
with open(test_file, encoding="utf-8") as file:
    text = file.readlines()

prompts = [t[:index_number].strip() for t in text]
numbers = [t[index_number:].strip() for t in text]

In [61]:
def get_accuracy(model, prompts: list, numbers: list) -> float:
    accuracy = 0
    activations = []
    for prompt, number in tqdm(zip(prompts, numbers), desc="Testing..."):
        input_ids = torch.tensor([tokenizer.encode(prompt)]).to(device)
        outputs_tokens = model.generate(input_ids, max_length=max_length, pad_token_id=tokenizer.eos_token_id)
        output_text = tokenizer.decode(outputs_tokens[0], skip_special_tokens=True)

        output_text = output_text[index_number:].strip()

        activation_ffn = model.get_activation_ffn()

        if output_text == number:
            activations.append(activation_ffn)
            accuracy += 1

    return accuracy * 100 / len(prompts), activations

def get_activation_ffn(model, prompts: list) -> list:
    activations = []
    for prompt in tqdm(prompts, desc="Getting activations..."):
        input_ids = torch.tensor([tokenizer.encode(prompt)]).to(device)
        _ = model.generate(input_ids, max_length=max_length, pad_token_id=tokenizer.eos_token_id)
        activation_ffn = model.get_activation_ffn()
        activations.append(activation_ffn)

    activation_ffn_array = activations

    return activation_ffn_array

In [62]:
acc, activations = get_accuracy(model, prompts, numbers)
print("Accuracy:", acc)

Testing...: 1000it [00:30, 32.37it/s]

Accuracy: 29.7


In [69]:
activation_ffn_array = np.stack(activations)

In [70]:
print(activation_ffn_array.shape)

(297, 12, 1, 1, 3072)


In [72]:
# reshape
activation_ffn_array = activation_ffn_array.reshape(num_layers, -1, num_neurons)
print(activation_ffn_array.shape)

(12, 297, 3072)


In [76]:
# save
with open("gpt2.pickle", 'wb') as f:
    pickle.dump(activation_ffn_array, f)

## Prunning

In [109]:
def get_low_activations_neuron_per_layer(activations: np.array, layer: int, threshold: float = 0.3, n_components: int = 10) -> tuple:
    first_layer_activations = activations[layer]
    
    scaler = MinMaxScaler()
    normalized_activations = scaler.fit_transform(first_layer_activations.reshape(-1, first_layer_activations.shape[-1])).reshape(first_layer_activations.shape)

    nmf = NMF(n_components=n_components, max_iter=1000, random_state=0)
    w = nmf.fit_transform(normalized_activations.reshape(-1, normalized_activations.shape[-1]))  # Aplana las activaciones para NMF
    h = nmf.components_
    
    low_contribution_neurons = np.where(h.mean(axis=0) < threshold)[0]

    return low_contribution_neurons, w, h

def plot_w_and_h_heatmaps(w: np.array, h: np.array, layer: int):
    # Mapa de calor para W
    plt.figure(figsize=(10, 8))
    sns.heatmap(w, cmap='viridis')
    plt.title('W Heatmap - Layer {}'.format(layer))
    plt.xlabel('Components')
    plt.ylabel('Tokens/Samples')
    plt.show()

    # Mapa de calor para H
    plt.figure(figsize=(10, 8))
    sns.heatmap(h, cmap='viridis')
    plt.title('H Heatmap - Layer {}'.format(layer))
    plt.xlabel('Neurons')
    plt.ylabel('Components')
    plt.show()

In [111]:
for i in range(num_layers):
    high_contribution_neurons, w, h = get_low_activations_neuron_per_layer(activation_ffn_array, i)
    print("Layer:", i, "Number of pruned neurons:", activation_ffn_array.shape[-1] - len(high_contribution_neurons))
    #plot_w_and_h_heatmaps(w, h, i) 

Layer: 0 Number of pruned neurons: 438
Layer: 1 Number of pruned neurons: 1559
Layer: 2 Number of pruned neurons: 492
Layer: 3 Number of pruned neurons: 1055
Layer: 4 Number of pruned neurons: 522
Layer: 5 Number of pruned neurons: 880
Layer: 6 Number of pruned neurons: 491
Layer: 7 Number of pruned neurons: 581
Layer: 8 Number of pruned neurons: 986
Layer: 9 Number of pruned neurons: 721
Layer: 10 Number of pruned neurons: 764
Layer: 11 Number of pruned neurons: 783
